In [1]:
import collections
import random
import pandas as pd
from tqdm import tqdm

def build_data_set(input_file_path, out_file_path, total_nums_of_rows=200000, dataset_list=[]):
    print(input_file_path)
    train_dict = collections.defaultdict(list)
    with open(input_file_path, encoding='utf-8') as tsvfile:
        reader = tsvfile.readlines()
        for i, row in enumerate(reader):
            row = row.strip().split("\t")
            id = int(row[0])
            instance = row[1]
            train_dict[id].append(instance)
        print("Total rows: %d" % i)
    print("Total ids: %d" % len(train_dict))
    df = pd.DataFrame()
    while len(dataset_list) < total_nums_of_rows:
        positive_id = random.choice(list(train_dict.keys()))
        negative_id = random.choice(list(train_dict.keys()))
        positive_sentence = train_dict[positive_id]
        negative_sentence = train_dict[negative_id]

        anchor = random.choice(positive_sentence)
        positive = random.choice(positive_sentence)
        negative = random.choice(negative_sentence)

        if anchor is None or positive is None or negative is None:
            raise('Error')

        dataset_list.append([anchor, positive, negative])

    df = pd.DataFrame((dataset_list))
    print(df.head())
    print(len(df))
    df.to_csv(out_file_path, index=False, sep='\t', header=None)
    print(out_file_path, ' Saved!')
    
    

In [2]:
filtered_file_path = 'data/v7/filter_v7/filtered_split_pair_all_siamese_bert_base_cased_epoch2.csv'
filtered_pairs = pd.read_csv(filtered_file_path, sep='\t')
trian_dict = collections.defaultdict(list)
for index, row in filtered_pairs.iterrows():
    if row[0] not in trian_dict:
        trian_dict[row[0]] = [[], []]

    if row[2] == 1:
        # Positive Pair
        trian_dict[row[0]][0].append(row[1])
    else:
        # Negative Pair
        trian_dict[row[0]][1].append(row[1])

In [3]:
filitered_triplet = []
for key in trian_dict:
    positive_list = trian_dict[key][0]
    negative_list = trian_dict[key][1]
    if len(positive_list) != len(negative_list):
        raise('Error')
    for positive, negative in zip(positive_list, negative_list):
        filitered_triplet.append([key, positive, negative])

print(len(filitered_triplet))

583136


In [4]:
build_data_set(input_file_path="data/v7/train_set_v1_7.txt",
               out_file_path="data/v7/triplet_train_filitered.csv",
               total_nums_of_rows=1200000,
               dataset_list=filitered_triplet)

data/v7/train_set_v1_7.txt
Total rows: 292747
Total ids: 9268
                                                   0  \
0  #2009faillist the New Jersey Nets, they should...   
1  #2009faillist the New Jersey Nets, they should...   
2  #2009faillist the New Jersey Nets, they should...   
3  #2009faillist the New Jersey Nets, they should...   
4  Major did you get Jake to unblock me? I have s...   

                                                   1  \
0    #cheatingexcuses I was taking one for the team.   
1  I cant believe the New Jersey Nets one a fucki...   
2  If he is driving an 89 Civic with 22 inch rims...   
3  #2009faillist guys with lip rings. Stop trying...   
4  I tweeted a funny joke at you when you were si...   

                                                   2  
0  #ihatewhen the #WhiteSox lose to rival teams l...  
1  #MW2 for the "boo". Can someone tell me why th...  
2  He should make that # famous. If I was in the ...  
3  #thingsilike seeing the Lakers beat the s